Copyright (C) 2023 Pablo Castells y Alejandro Bellogín

El código que contiene este notebook se ha implementado para la realización de las prácticas de la asignatura "Búsqueda y minería de información" de 4º del Grado en Ingeniería Informática, impartido en la Escuela Politécnica Superior de la Universidad Autónoma de Madrid. El fin del mismo, así como su uso, se ciñe a las actividades docentes de dicha asignatura.

### **Búsqueda y Minería de Información 2022-23**
### Universidad Autónoma de Madrid, Escuela Politécnica Superior
### Grado en Ingeniería Informática, 4º curso

# Bloque 1 &ndash; Sistemas de recomendación

Fechas:

* Comienzo: martes 28 / jueves 30 de marzo
* Entrega: lunes 8 de mayo, 23:59

## Objetivos

Este primer bloque de la práctica tiene por objetivo la implementación y evaluación eficiente de sistemas de recomendación. En este bloque se desarrollarán:

* Estructuras para el manejo de datos de interacción entre usuarios e items ("ratings" para simplificar).
* Algoritmos de recomendación basada en filtrado colaborativo.
* Métricas de evaluación de sistemas de recomendación.

## Material proporcionado

Se proporcionan software y datos para la realización de la práctica:

* Un esqueleto de clases y funciones donde el estudiante desarrollará sus implementaciones. 
  - De modo similar a las prácticas anteriores, se proporciona una celda de prueba al final de este notebook que deberá funcionar con las implementaciones del estudiante.
  - Junto a la celda de prueba en este mismo notebook, se muestra como referencia un ejemplo de salida generada con una implementación de los profesores.
* Los siguientes conjuntos de datos de ratings por usuarios a items:
  - Dos conjuntos de juguete para prueba y depuración: <ins>toy1.csv</ins> (se genera en Matrices.ipynb) y <ins>toy2.csv</ins> (proporcionado en el curso Moodle) con ratings ficticios.
  - Un conjunto de datos reales de ratings a películas: *ml-1m.zip* disponible en la Web de [MovieLens](https://grouplens.org/datasets/movielens/1m). De los archivos disponibles, se utilizará sólamente <ins>ratings.dat</ins>, añadiéndole una cabecera `u::i::r::t`.
  
Los esqueletos de código que se proporcionan aquí son a modo de guía: el estudiante puede modificarlo todo libremente, siempre que la celda de prueba funcione correctamente **sin cambios**.

## Calificación

El peso de esta práctica (recomendación + redes sociales) en la nota final de prácticas es del **40%**.

La calificación se basará en el **número** de ejercicios realizados y la **calidad** de los mismos. La puntuación que se indica en cada apartado es orientativa, en principio se aplicará tal cual se refleja pero podrá matizarse por criterios de buen sentido si se da el caso.

Para dar por válida la realización de un ejercicio, el código deberá funcionar (a la primera) integrado con las clases que se facilitan. El profesor comprobará este aspecto ejecutando la celda de prueba y otras adicionales.

La corrección de las implementaciones se observará por la **coherencia de los resultados** (por ejemplo, las métricas sobre los algoritmos de recomendación), y se valorará la eficiencia en tiempo de ejecución.

## Entrega

La entrega consistirá en dos ficheros tipo *notebook* (uno para recomendación y otro para redes sociales) donde se incluirán todas las **implementaciones** solicitadas en cada ejercicio, así como una explicación de cada uno a modo de **memoria**.

## Indicaciones

La realización de los ejercicios conducirá en muchos casos a la implementación de funciones y/o clases adicionales a las que se indican en el enunciado. Algunas vendrán dadas por su aparición en los propias celdas de prueba, y otras por conveniencia a criterio del estudiante.

Igual que en prácticas anteriores, no deberán editarse las celdas de prueba. Estas celdas deberán ejecutar sin errores a la primera con el código entregado por el estudiante (naturalmente con salvedad de los ejercicios que no se hayan implementado.

## Autores

Xu Chen Xu <br>
Ana Martínez Sabiote

## Ejercicio 1: Estructuras de datos y recomendación simple (2pt)

#### 1.1 &nbsp; Estructuras de datos

Implementar las clases necesarias para manejar **datos de entrada y prueba** (ratings) para los algoritmos de recomendación. La funcionalidad se implementará en una clase Ratings, que permitirá leer los datos de un fichero de texto, así como un método que genere dos particiones aleatorias de entrenamiento y test, para evaluar y comparar la efectividad de diferentes algoritmos de recomendación.

In [1]:
import numpy as np
import pandas as pd

class Ratings:
    def __init__(self, file=None, sep=',', df=None):
        # Código aquí...
        if file is not None:
            self.ratings_df=pd.read_csv(filepath_or_buffer=file, sep = sep)
        elif df is not None:
            self.ratings_df=df
        else:
            raise Exception("No se ha especificado un fichero ni un dataframe")

        self.uids=self.ratings_df.u.unique()
        self.iids=self.ratings_df.i.unique()
        matrix = self.ratings_df.pivot(index='u', columns='i', values='r')
        matrix = matrix.fillna(0)
        self.m = matrix.to_numpy()

        self.uidx_to_uid_dict = np.sort(self.ratings_df.u.unique())
        self.iidx_to_iid_dict = np.sort(self.ratings_df.i.unique())
        self.uid_to_uidx_dict = {u:j for j, u in enumerate(self.uidx_to_uid_dict)}
        self.iid_to_iidx_dict = {i:j for j, i in enumerate(self.iidx_to_iid_dict)}

    def matrix(self):
        # Código aquí...
        return self.m

    def nusers(self):
        return self.m.shape[0]

    def nitems(self):
        # Código aquí...
        return self.m.shape[1]

    # uidx can be an int or an array-like of ints.
    def uidx_to_uid(self, uidx):
        # Código aquí...
        return self.uidx_to_uid_dict[uidx]

    # iidx can be an int or an array-like of ints.
    def iidx_to_iid(self, iidx):
        # Código aquí...
        return self.iidx_to_iid_dict[iidx]

    # uid can be an int or an array-like of ints.
    def uid_to_uidx(self, uid):
        # Código aquí...
        return self.uid_to_uidx_dict[uid]

    # iid can be an int or an array-like of ints.
    def iid_to_iidx(self, iid):
        # Código aquí...
        return self.iid_to_iidx_dict[iid]

    def iidx_rated_by(self, uidx):
        # Código aquí...
        # Distintos de cero por filas
        positions = np.where(self.m[uidx,:]!=0)
        return positions

    def uidx_who_rated(self, iidx):
        # Código aquí...
        # Distintos de cero por columnas
        positions = np.where(self.m[:,iidx]!=0)
        return positions


    def random_split(self, ratio):
        # Código aquí...
        # Devuelve dos objetos ratings
        # Usar pandas.DataFrame.Sample sobre self.ratings_df para obtener los dataframes 
        # train y test y crear un objeto Ratings a partir de esos dataframes. 

        # Dividimos el dataframe en dos partes
        mask_1 = np.random.choice([1,0], size=self.matrix().shape, p=[ratio,1-ratio])
        training = self.matrix()*mask_1

        mask_2 = 1-mask_1
        test = self.matrix()*mask_2

        df1=pd.DataFrame(training, index=self.uidx_to_uid_dict, columns=self.iidx_to_iid_dict)
        df2=pd.DataFrame(test, index=self.uidx_to_uid_dict, columns=self.iidx_to_iid_dict) 
        df1=df1.unstack().reset_index(name='r')
        df1.columns = ['i', 'u', 'r']
        df2=df2.unstack().reset_index(name='r')
        df2.columns = ['i', 'u', 'r']


        #df1 = self.ratings_df.sample(frac=ratio)
        # CORREGIR df2: ES NECESARIO  QUE df1 Y df2 TENGAN LA MISMA DIMENSIÓN
        #df2 = self.ratings_df.loc[~self.ratings_df.index.isin(df1.index)]


        # Creamos los objetos Ratings
        ratings1 = Ratings(df=df1)
        ratings2 = Ratings(df=df2)

        return ratings1, ratings2

    #
    # The remaining functions are just for debugging purposes
    #

    def rating(self, uid, iid):
        # Código aquí...
        uidx=self.uid_to_uidx(uid)
        iidx=self.iid_to_iidx(iid)
        return self.m[uidx,iidx]

    def items_rated_by(self, uid):
        # Código aquí...2
        uidx=self.uid_to_uidx(uid)
        return self.iidx_to_iid(self.iidx_rated_by(uidx))

    def users_who_rated(self, iid):
        # Código aquí...
        iidx=self.iid_to_iidx(iid)
        return self.uidx_to_uid(self.uidx_who_rated(iidx))

    def user_ratings(self, uid):
        # Código aquí...
        uidx=self.uid_to_uidx(uid)
        positions=np.where(self.m[uidx,:]!=0)
        ratings=self.m[uidx, positions]
        return dict(zip(self.items_rated_by(uid),ratings[0,:]))

    def item_ratings(self, iid):
        # Código aquí...
        iidx=self.iid_to_iidx(iid)
        positions=np.where(self.m[:,iidx]!=0)
        ratings=self.m[positions,iidx]
        return dict(zip(self.users_who_rated(iid),np.concatenate(ratings)))
        

    def nratings(self):
        # Código aquí...
        return np.count_nonzero(self.m)

    # To inspect random data splits.
    def save(self, file):
        df = pd.DataFrame(columns=self.iids, index=self.uids, data=self.m).unstack().reset_index(name='r')
        df.columns = ['i', 'u', 'r']
        df = df[df.r>0][['u', 'i', 'r']].sort_values(by=['u', 'i'])
        df.to_csv(file, index=False)

# Just for pretty-printing numbers.
def fround(x, n=20):
    r = round(x)
    rn = round(x, n)
    return r if rn == r else rn

In [2]:
# example array
a = np.array([[1, 0, 0], [0, 0, 6]])

# a to one dimension
a = a.flatten()

# get the indexes of the non-zero values
idx = np.nonzero(a)

# Select 0.8 proportions of the indexes
idx_1 = np.random.choice(idx[0], size=int(0.8*len(idx[0])), replace=False)

# Create a mask of zeros and ones
mask = np.zeros(len(a))
mask[idx_1] = 1

# Apply the mask to the array
a = a*mask

# Reshape the array to the original shape
a = a.reshape(2,3)

# Print the array
print(a)



[[0. 0. 0.]
 [0. 0. 6.]]


In [3]:
ratings = Ratings('recsys-data/toy1.csv')

m = ratings.matrix()

m

array([[0., 4., 5., 3., 0.],
       [5., 2., 0., 0., 4.],
       [1., 4., 4., 0., 0.],
       [0., 0., 3., 5., 0.]])

In [4]:
ratings = Ratings('recsys-data/toy1.csv')

proportion = 0.8
m = ratings.matrix()

# Create matrix with same size as m but filled with zeros
m_zeros = np.zeros(m.shape)

# In the positions where m > 0, put a random number
m_zeros[m>0] = np.random.rand(np.count_nonzero(m))

m_zeros > proportion
# train, test = ratings.random_split(0.8)

# display(pd.DataFrame(ratings.matrix(), index=ratings.uidx_to_uid_dict, columns=ratings.iidx_to_iid_dict))
# display(pd.DataFrame(train.matrix(), index=train.uidx_to_uid_dict, columns=train.iidx_to_iid_dict))
# display(pd.DataFrame(test.matrix(), index=test.uidx_to_uid_dict, columns=test.iidx_to_iid_dict))

array([[False, False, False,  True, False],
       [False,  True, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False]])

#### 1.2 &nbsp;  Recomendaciones: métodos simples no personalizados

La **salida** de un recomendador consistirá en un diccionario con un ránking por usuario. 

Implementar un primer **recomendador simple** por rating promedio en una clase `AverageRecommender`. El recomendador sólo recomendará items que tengan un mínimo número de ratings, que se indicará como parámetro en el constructor (con ello se mejora el acierto de la recomendación). Se proporciona una clase `MajorityRecommender` a modo de ejemplo en el que el estudiante podrá basarse, así como `RandomRecommender`, que se utiliza en ocasiones como referencia en experimentos. 

**Importante**: recordar que no deben recomendarse los items que los usuarios ya hayan puntuado.

In [5]:
# Ejemplos míos para entender las funciones de la celda siguiente.

def top_positions_per_row(m, k):
    return np.sort(np.argpartition(m, -k)[:, -k:], axis=1)[:, ::-1]

def get_elements(m, indices, cutoff=np.inf):
    return np.array([s[t[0:min(cutoff, len(t))]] for s, t in zip(m, indices)])

matrix = np.array([[1, 2, 3, 4, 5],[3,4,5,6,7], [5,6,7,8,9]])

print(top_positions_per_row(matrix, 3))

print(get_elements(matrix, [[0,2],[0,2]]))

aux = np.array([[0,4],[0,2]])

print(get_elements(matrix, aux))

[[4 3 2]
 [4 3 2]
 [4 3 2]]
[[1 3]
 [3 5]]
[[1 5]
 [3 5]]


In [6]:
# Suggestion: compute the scores in the recommenders' constructor.

from itertools import islice

# Given a matrix, returns a matrix of positions of top k values per row.
def top_positions_per_row(m, k):
    return np.sort(np.argpartition(m, -k)[:, -k:], axis=1)[:, ::-1]

# Given a matrix and a set of indices per rows, returns the matrix values for the indices. 
# This function is used in the Recommender class and metrics classes.
def get_elements(m, indices, cutoff=np.inf):
    return np.array([s[t[0:min(cutoff, len(t))]] for s, t in zip(m, indices)])

class Recommendation:
    def __init__(self, scores, n, training):
        # Código aquí...
        # Indicación: generar aquí los ránkings de scores, de iidx's y de iids.
        self.scores = scores

        self.top_iidx = top_positions_per_row(scores, n)
        # Sort because top_positions_per_row returns the top unsorted.
        self.ranked_iidx = get_elements(self.top_iidx, np.argsort(get_elements(scores, self.top_iidx))[:, ::-1])

        # And now get the ranked uids and scores.
        ranked_iids = training.iidx_to_iid(self.ranked_iidx)
        rank_scores = get_elements(scores, self.ranked_iidx)

        self._recommendation = {uid : [(iid, score) for iid, score in zip(ranked_iids[uidx], rank_scores[uidx]) if score > 0]
                for uidx, uid in enumerate(training.uidx_to_uid_dict)}

    def ranked_iidx(self):
        # Código aquí...
        return self.ranked_iidx

    def recommendation(self, uid):
        # Código aquí...
        return self._recommendation[uid]

    # Format the recommendation as a string for the first n users. Trim scores to 4 decimal digits.
    def display(self, n):
        r = ''
        for uid in islice(self._recommendation, n):
            r += f'    User {uid} -> <' 
            for iid, score in self.recommendation(uid): 
                r += f'{iid}:' + str(fround(score, 4)) + ' '
            r = (r[:-1] + '>\n') if len(self.recommendation(uid)) > 0 else r + 'empty>\n'
        return r[:-1]

class Recommender():
    def __init__(self, training):
        self.training = training

    def __repr__(self):
        return type(self).__name__

    def recommend(self, n):
        return Recommendation(self.scores, n, self.training)

class RandomRecommender(Recommender):
    def __init__(self, training):
        super().__init__(training)
        self.scores = np.random.random(training.matrix().shape)

class MajorityRecommender(Recommender):
    def __init__(self, training, threshold=0):
        super().__init__(training)
        # training.matrix() >= threshold creates a mask with 'True' on relevant ratings and 'False' anywhere
        # else. Thus 'pop' is an array with the counts of relevant ratings of each item.

        pop = np.sum(training.matrix() >= threshold, axis=0)

        # This product by a vector of ones (of user-row length) creates a matrix where the pop vector gets
        # copied on all rows; the recommendation is not personalized and ranking is the same for all users 
        # -- except of course in the end different training items will be filtered out for different users.
        self.scores = np.outer(np.ones(training.nusers()), pop)

class AverageRecommender(Recommender):
    def __init__(self, training, minr=0):
        super().__init__(training)
        self.minr = minr

        not_rated_mask = (training.matrix() == 0)

        # Calculamos la suma de los ratings para cada item
        sum_ratings = training.matrix().sum(axis=0)

        # Calculamos el número de ratings de cada item
        n_ratings = (training.matrix() > 0).sum(axis=0)

        mean_ratings = np.divide(sum_ratings, n_ratings, 
                                 out=np.zeros_like(sum_ratings), where = (n_ratings!=0))

        # Creamos un vector con la media de los ratings de cada item
        # si tiene por lo menos minr ratings. Si tiene menos, la media es 0.
        mean_ratings = np.where(n_ratings >= minr, mean_ratings, 0)

        # Creamos la matriz de scores donde cada score es la media de los ratings
        self.scores = np.outer(np.ones(training.nusers()), mean_ratings)

        # Nos quedamos solo con los scores de los items que no han sido valorados por cada usuario
        self.scores = self.scores * not_rated_mask

In [7]:
# Celda de testing
ratings = Ratings('recsys-data/toy1.csv', sep=',')

ratings.matrix()

# Create df from np array
display(pd.DataFrame(ratings.matrix(), index=ratings.uidx_to_uid_dict, columns=ratings.iidx_to_iid_dict))

# Create AverageRecommender
mr = AverageRecommender(ratings, 2)

display(pd.DataFrame(mr.scores, index=ratings.uidx_to_uid_dict, columns=ratings.iidx_to_iid_dict))


,a,b,c,d,e
v,0.0,4.0,5.0,3.0,0.0
x,5.0,2.0,0.0,0.0,4.0
y,1.0,4.0,4.0,0.0,0.0
z,0.0,0.0,3.0,5.0,0.0


,a,b,c,d,e
v,3.0,0.000000,0.0,0.0,0.0
x,0.0,0.000000,4.0,4.0,0.0
y,0.0,0.000000,0.0,4.0,0.0
z,3.0,3.333333,0.0,0.0,0.0


In [8]:
a=np.array([[2,3,4],[2,0,4],[3,3,3],[0,0,1]])
resultado=np.where(a[1,:]!=0)
print(resultado)
print(a[resultado])

(array([0, 2], dtype=int64),)
[[2 3 4]
 [3 3 3]]


In [9]:
# Celda de testing
ratings = Ratings('recsys-data/toy1.csv', sep=',')

rec = RandomRecommender(ratings)
rec = rec.recommend(3)
display(rec.ranked_iidx)
display(rec._recommendation)
print(rec.display(3))

array([[0, 3, 2],
       [1, 2, 3],
       [4, 2, 0],
       [1, 0, 3]], dtype=int64)

{'v': [('a', 0.9858277478168467),
  ('d', 0.9374213594623059),
  ('c', 0.7627218681945643)],
 'x': [('b', 0.9369089199558277),
  ('c', 0.7257347699656793),
  ('d', 0.5937670613617214)],
 'y': [('e', 0.859549759257613),
  ('c', 0.5581464914135078),
  ('a', 0.4788612704595886)],
 'z': [('b', 0.8464838392835399),
  ('a', 0.5745280037011418),
  ('d', 0.3294377427883902)]}

    User v -> <a:0.9858 d:0.9374 c:0.7627>
    User x -> <b:0.9369 c:0.7257 d:0.5938>
    User y -> <e:0.8595 c:0.5581 a:0.4789>


In [10]:
training,test=ratings.random_split(0.8)
print(ratings.matrix())
print(training.matrix())
print(test.matrix())

[[0. 4. 5. 3. 0.]
 [5. 2. 0. 0. 4.]
 [1. 4. 4. 0. 0.]
 [0. 0. 3. 5. 0.]]
[[0. 4. 0. 3. 0.]
 [5. 2. 0. 0. 4.]
 [0. 4. 0. 0. 0.]
 [0. 0. 3. 5. 0.]]
[[0. 0. 5. 0. 0.]
 [0. 0. 0. 0. 0.]
 [1. 0. 4. 0. 0.]
 [0. 0. 0. 0. 0.]]


### Ejercicio 1 &ndash; Explicación/documentación

(por hacer)

## Ejercicio 2: Filtrado colaborativo kNN (2pt)

Implementar un algoritmo de filtrado colaborativo mediante vecinos próximos orientado a usuarios por *similitud coseno* (sin normalizar por la suma de similitudes). 

In [11]:
class CosineUserSimilarity:
    def __init__(self, training):
        self.training = training
        ratings_matrix = training.matrix()

        dots = ratings_matrix @ ratings_matrix.T
        mods = np.sqrt(dots.diagonal())

        # To avoid 0/0 later if some row is all zeros (and hence the row modulus is zero).
        mods[mods==0] = 1

        # Vamos ahora a dividir por los modulos
        # Dividimos cada fila de dots por cada elemento de mods
        # Es decir, primera fila de mods por primer elemento de mods,...
        self.sim = dots/mods[:, None]

        # Dividimos cada columna de sim por cada elemento de mods.
        # Es decir, primera columna de sim por primer elemento de mods,...
        self.sim = self.sim/mods

        # Ponemos a 0 la diagonal porque no queremos que un usuario sea similar a si mismo
        np.fill_diagonal(self.sim,0)

    def similarity_matrix(self):
        return self.sim

class UserKNNRecommender(Recommender):
    def __init__(self, training, sim, k):
        super().__init__(training)
        self.sim = sim
        self.k = k

        not_rated_mask = (training.matrix() == 0)

        # Indices de los k usuarios mas similares a cada usuario
        uidx = top_positions_per_row(sim.similarity_matrix(), k)

        # Mascara con 1s en las posiciones de los k usuarios mas similares a cada usuario
        # y 0s en el resto
        top_similar_mask = np.zeros_like(sim.similarity_matrix())
        top_similar_mask[np.arange(top_similar_mask.shape[0]), uidx.T] = 1

        # Matriz de similitudes entre los k usuarios mas similares a cada usuario.
        # El resto es 0.
        knn_sim = sim.similarity_matrix() * top_similar_mask

        # Creamos la matriz de scores donde cada score es la media ponderada (segun la similitud entre usuarios)
        # de los ratings de los k usuarios mas similares
        self.scores = knn_sim @ training.matrix()

        # Nos quedamos solo con los scores de los items que no han sido valorados por cada usuario
        self.scores = self.scores * not_rated_mask

In [12]:
# Celda de testing (aún no he mirado que este bien)
# Ana: confirmo que comparándolo con el ejemplo de P3-Matrices.ipynb SÍ funciona correctamente
rec = UserKNNRecommender(ratings, CosineUserSimilarity(ratings), 2)
recommendation = rec.recommend(3)

display(recommendation._recommendation)

recommendation.recommendation('x')

{'v': [('a', 0.8862587350511957)],
 'x': [('c', 2.1926722125915408), ('d', 0.5059644256269407)],
 'y': [('d', 4.450020507233184)],
 'z': [('b', 4.343422942099672), ('a', 0.35824886041591925)]}

[('c', 2.1926722125915408), ('d', 0.5059644256269407)]

In [13]:
print(recommendation.scores)
mask_cutoff=(recommendation.scores>4.0)

mask_test=(test==0)
scores=recommendation.scores*mask_cutoff
print(scores)
scores=(scores>0).astype(int)
print(scores)
pk=[]
scores=(scores>0).astype(int)
# Vamos sumando las filas y dividiendo por k
for i in range(scores.shape[1]):
    pk.append(np.sum(scores[:,i])/i)
# Sumamos por columnas y dividimos por el número de filas, es decir, de usuarios    
result=np.sum(pk,axis=0)/scores.shape[1]
print(result)

[[0.88625874 0.         0.         0.         0.        ]
 [0.         0.         2.19267221 0.50596443 0.        ]
 [0.         0.         0.         4.45002051 0.        ]
 [0.35824886 4.34342294 0.         0.         0.        ]]
[[0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         4.45002051 0.        ]
 [0.         4.34342294 0.         0.         0.        ]]
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]
 [0 1 0 0 0]]
nan


C:\Users\anama\AppData\Local\Temp/ipykernel_17080/1229805489.py:13: RuntimeWarning: invalid value encountered in long_scalars
  pk.append(np.sum(scores[:,i])/i)


### Ejercicio 2 &ndash; Explicación/documentación

(por hacer)

## Ejercicio 3: Evaluación (1pt)

Se desarrollarán clases que permitan calcular métricas para evaluar y comparar el acierto de los recomendadores: se implementarán **precisión** y **recall**. 

Como resumen de este bloque, se incluirá una *tabla con los valores de las métricas* (dos columnas) más el tiempo de ejecución (una columna más) sobre todos los algoritmos implementados (filas), al menos para el conjunto de datos de <ins>MovieLens 1M</ins>. En el caso de ser capaces de procesar un conjunto de datos más grande, se documentará el tamaño en RAM de la matriz de ratings.

<!-- Opcionalmente, se podrán implementar otras métricas a elección del estudiante (nDCG, etc.), cuya prueba se incluirá en la función `student_test()` del ejercicio 4 ("ampliaciones"). -->

In [14]:
class Metric():
    def __init__(self, test, cutoff):
        # Objeto raitings que contiene la partición de test
        self.test = test
        self.cutoff = cutoff

    def __repr__(self):
        return type(self).__name__ + ('@' + str(self.cutoff) if self.cutoff != np.inf else '')

class Precision(Metric):
    def __init__(self, test, cutoff=np.inf, threshold=1):
        super().__init__(test, cutoff)
        # Código aquí...
        # threshold debe ser el umbral de relevancia
        self.threshold=threshold
    
        

    def compute(self, recommendation):
        # Código aquí...  
        # Nos quedamos con los relevantes
        # COGER TOP cutoff
        top_iidx = top_positions_per_row(recommendation.scores, self.cutoff)
        # Sort because top_positions_per_row returns the top unsorted.
        ranked_iidx = get_elements(top_iidx, np.argsort(get_elements(recommendation.scores, top_iidx))[:, ::-1])
        rank_scores = get_elements(recommendation.scores, ranked_iidx)

        # Tenemos en cuenta los relevantes de los scores (ya ordenados top cutoff)
        mask_threshold=(rank_scores>self.threshold)
        # Ordenamos test para que concuerde uid y iid con el orden de rank_scores (top cutoff)
        mask_test=np.take_along_axis(self.test.matrix(),ranked_iidx, axis=1)
        # Tenemos en cuenta los relevantes del test
        mask_test=(mask_test>self.threshold)
        
        """
        print(rank_scores)
        print(mask_threshold)
        print(mask_test)
        """
        
        scores=mask_threshold*mask_test
        
        # Cálculos para precisión
        pk=np.sum(scores, axis=1)/self.cutoff
        # Por último promediamos la precisión con el número de usuarios
        result=np.sum(pk)/len(pk)
        
        return result
        
        
class Recall(Metric):
    def __init__(self, test, cutoff=np.inf, threshold=1):
        super().__init__(test, cutoff)
        # Código aquí...
        self.threshold=threshold

    def compute(self, recommendation):
        # Código aquí...
        # Nos quedamos con los TOP cutoff
        top_iidx = top_positions_per_row(recommendation.scores, self.cutoff)
        # Sort because top_positions_per_row returns the top unsorted.
        ranked_iidx = get_elements(top_iidx, np.argsort(get_elements(recommendation.scores, top_iidx))[:, ::-1])
        rank_scores = get_elements(recommendation.scores, ranked_iidx)

        # Tenemos en cuenta los relevantes de los scores (ya ordenados top cutoff)
        mask_threshold=(rank_scores>self.threshold)
        # Ordenamos test para que concuerde uid y iid con el orden de rank_scores (top cutoff)
        mask_test=np.take_along_axis(self.test.matrix(),ranked_iidx, axis=1)
        # Tenemos en cuenta los relevantes del test
        mask_test=(mask_test>self.threshold)
        """
        print(recommendation.ranked_iidx)
        print(mask_cutoff)
        print(mask_test)
        print(self.test.matrix())
        """
    
        scores=mask_threshold*mask_test
        
        
        # Cálculos para recall
        # Relevantes por usuario
        # Relevantes totales test
        relevante_total_test=(self.test.matrix()>self.threshold)
        relevantes=np.sum(relevante_total_test,axis=1)
        # Para solucionar divisiones por cero cuando no hay resultados relevantes para un usuario
        relevantes[relevantes == 0]=1
        # Para calcular recall de cada usuario dividimos
        # número de relevantes TOP cutoff entre número de relevantes totales
        pk=np.sum(scores, axis=1)/relevantes
        # Por último promediamos el recall con el número de usuarios
        result=np.sum(pk)/len(pk)
        
        return result

### Ejercicio 3 &ndash; Explicación/documentación

(por hacer)

Ejemplo de tabla de resumen:

||Precision@K|Recall@K|Tiempo de ejecución
|-|:-:|:-:|:-:
|Algoritmo 1|...|...|...
|Algoritmo 2|...|...|...
|...|...|...|...
|Algoritmo n|...|...|...

## Ejercicio 4: Ampliaciones (1pt)

Elegir uno de los siguientes ejercicios:

* Implementar dos variantes de kNN a elección del estudiante, por ejemplo: kNN normalizado, vecinos próximos orientado a item, similitud de Pearson, kNN centrado en la media. Indicación: para kNN normalizado el algoritmo exigirá un mínimo de ratings de vecinos para aceptar recomendar un item (con ello se mejora el acierto de la recomendación, de forma similar a la recomendación por rating promedio).
* Implementar filtrado colaborativo mediante factorización de matrices.
* Crear una implementación de las estructuras de ratings con matrices dispersas, de forma que sea posible generar recomendaciones sobre conjuntos de datos más grandes, tales como [MovieLens 10M](https://grouplens.org/datasets/movielens/10m) y [MovieLens 25M](https://grouplens.org/datasets/movielens/25m).

Para probar las implementaciones deberá completarse la función `student_test()` para ilustrar la ejecución de las variantes adicionales, y se incluirán las filas que correspondan en la tabla del apartado anterior.

In [58]:
# Código aquí: clases, funciones...
# kNN normalizado
class UserKNNRecommenderNormalized(Recommender):
    def __init__(self, training, sim, k, k_min):
        super().__init__(training)
        self.sim = sim
        self.k = k

        not_rated_mask = (training.matrix() == 0)

        # Indices de los k usuarios mas similares a cada usuario
        uidx = top_positions_per_row(sim.similarity_matrix(), k)
        print(uidx.shape)
        mask=np.zeros_like(training.matrix())
        mask[uidx,np.arange(mask.shape[1])]=1
       
        print("Mask")
        print(mask)

        # Mascara con 1s en las posiciones de los k usuarios mas similares a cada usuario
        # y 0s en el resto
        top_similar_mask = np.zeros_like(sim.similarity_matrix())
        top_similar_mask[np.arange(top_similar_mask.shape[0]), uidx.T] = 1

        # Matriz de similitudes entre los k usuarios mas similares a cada usuario.
        # El resto es 0.
        knn_sim = sim.similarity_matrix() * top_similar_mask
        
        ####
        #mean_ratings = np.divide(sum_ratings, n_ratings, out=np.zeros_like(sum_ratings), where = (n_ratings!=0))

        # Creamos un vector con la media de los ratings de cada item
        # si tiene por lo menos minr ratings. Si tiene menos, la media es 0.
        #mean_ratings = np.where(n_ratings >= k_min, mean_ratings, 0)
        
        # Norma de cada item
        mask_scores=(training.matrix()!=0)
        norms=knn_sim @ mask_scores
        print(mask_scores)
        print(sim.similarity_matrix())
        print(norms)

        # Creamos la matriz de scores donde cada score es la media ponderada (segun la similitud entre usuarios)
        # de los ratings de los k usuarios mas similares
        knn_scores=knn_sim @ training.matrix()
        # Normalizamos
        self.scores = np.divide(knn_scores, norms,out=np.zeros_like(knn_scores), where = (norms!=0))

        # Nos quedamos solo con los scores de los items que no han sido valorados por cada usuario
        self.scores = self.scores * not_rated_mask
        
        #self.scores = np.where(n_ratings >= minr, mean_ratings, 0)
        
class CosineItemSimilarity:
    def __init__(self, training):
        self.training = training
        ratings_matrix = training.matrix()

        dots = ratings_matrix.T @ ratings_matrix
        mods = np.sqrt(dots.diagonal())

        # To avoid 0/0 later if some row is all zeros (and hence the row modulus is zero).
        mods[mods==0] = 1

        # Vamos ahora a dividir por los modulos
        # Dividimos cada fila de dots por cada elemento de mods
        # Es decir, primera fila de mods por primer elemento de mods,...
        self.sim = dots/mods[:, None]

        # Dividimos cada columna de sim por cada elemento de mods.
        # Es decir, primera columna de sim por primer elemento de mods,...
        self.sim = self.sim/mods

        # Ponemos a 0 la diagonal porque no queremos que un item sea similar a si mismo
        np.fill_diagonal(self.sim,0)

    def similarity_matrix(self):
        return self.sim

class ItemKNNRecommender(Recommender):
    def __init__(self, training, sim, k):
        super().__init__(training)
        self.sim = sim
        self.k = k

        not_rated_mask = (training.matrix() == 0)

        # Indices de los k usuarios mas similares a cada usuario
        iidx = top_positions_per_row(sim.similarity_matrix(), k)

        # Mascara con 1s en las posiciones de los k usuarios mas similares a cada usuario
        # y 0s en el resto
        top_similar_mask = np.zeros_like(sim.similarity_matrix())
        top_similar_mask[np.arange(top_similar_mask.shape[0]), iidx.T] = 1

        # Matriz de similitudes entre los k usuarios mas similares a cada usuario.
        # El resto es 0.
        knn_sim = sim.similarity_matrix() * top_similar_mask

        # Creamos la matriz de scores donde cada score es la media ponderada (segun la similitud entre usuarios)
        # de los ratings de los k usuarios mas similares
        self.scores = knn_sim @ training.matrix()

        # Nos quedamos solo con los scores de los items que no han sido valorados por cada usuario
        self.scores = self.scores * not_rated_mask
        

def student_test():
    # Código de prueba aquí...
    ratings = Ratings('recsys-data/toy1.csv')
    train, test = ratings.random_split(0.3)
    sim = CosineUserSimilarity(train)
    k=4
    knn = UserKNNRecommenderNormalized(train, sim, k,2)
    recommendation = knn.recommend(3)
    display(recommendation._recommendation)
    print(recommendation.display(3))

In [59]:
student_test()

(4, 4)


IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (4,4) (5,) 

### Ejercicio 4 &ndash; Explicación/documentación

(por hacer)

In [ ]:
pip install termcolor

Note: you may need to restart the kernel to use updated packages.


## Celda de prueba

Descarga los ficheros de datos y coloca sus contenidos en una carpeta **recsys-data** en el mismo directorio que este *notebook*.

In [55]:
import datetime
import time

# Test data structures and algorithms on a dataset.
def test(ratings_file, example_user, example_item, k, minr, topn=np.inf, cutoff=np.inf, threshold=1, sep=','):
    print(colored(f'Reading the data at ' + time.strftime('%X...'), 'blue'))
    start = time.time()
    ratings = Ratings(ratings_file, sep)
    print(f'Ratings matrix takes {round(10 * ratings.matrix().nbytes / 1024 / 1024) / 10:,} MB in RAM')
    timer(start)

    # Test Ratings class on the dataset.
    test_data(ratings, example_user, example_item)
    
    # Produce a rating split and test a set of recommenders. 
    train, test = ratings.random_split(0.8)
    metrics = [Precision(test, cutoff=cutoff, threshold=threshold), Recall(test, cutoff=cutoff, threshold=threshold)]
    run_recommenders(train, metrics, k, minr, topn)

# Test the rating data handling code (Ratings class).
def test_data(ratings, example_user, example_item):
    print('-------------------------\nTesting the ratings data structures')
    print(f'{ratings.nratings():,} ratings by {ratings.nusers():,} users on {ratings.nitems():,} items')
    print(f'Ratings of user {example_user}: {ratings.user_ratings(example_user)}')
    print(f'Ratings of item {example_item}: {ratings.item_ratings(example_item)}')

# Run some recommenders on the some rating data as input - no evaluation.
def run_recommenders(train, metrics, k, minr, topn):
    print('-------------------------')
    start = time.time()
    run_recommender(RandomRecommender(train), metrics, topn)
    start = timer(start)
    
    print('-------------------------')
    run_recommender(MajorityRecommender(train, threshold=4), metrics, topn)
    start = timer(start)
    
    print('-------------------------')
    run_recommender(AverageRecommender(train, minr), metrics, topn)
    start = timer(start)
    
    print('-------------------------')
    print('Creating user cosine similarity')
    sim = CosineUserSimilarity(train)
    start = timer(start)
    # print('-------------------------')
    print('Creating kNN recommender')
    knn = UserKNNRecommender(train, sim, k)
    start = timer(start)
    run_recommender(knn, metrics, topn)
    timer(start)
    
    # print('-------------------------')
    # start = time.time()
    # print('Creating MF recommender')
    # mf = MF(train, dim=20, lrate=.0001, reg=.005, nepochs=50)
    # timer(start)
    # start = time.time()
    # run_recommender(mf, metrics, topn)
    # timer(start)

# Run a recommender and evaluate a list of metrics on its output.
def run_recommender(recommender, metrics, topn):
    print(f'Testing {recommender} (top {topn})')
    recommendation = recommender.recommend(topn)
    print('Four example recommendations:\n' + recommendation.display(4))
    for metric in metrics:
        print(metric, '=', metric.compute(recommendation))

from termcolor import colored
def timer(start):
    print(colored(f'--> elapsed time: {datetime.timedelta(seconds=round(time.time() - start))} <--', 'blue'))
    return time.time()
    
np.random.seed(0)
print('=========================\nTesting toy 1 dataset')
test('recsys-data/toy1.csv', example_user='v', example_item='b', k=4, minr=2, topn=4, cutoff=4)
print('=========================\nTesting toy 2 dataset')
test('recsys-data/toy2.csv', example_user=1, example_item=2, k=4, minr=2, topn=4, cutoff=4)
print('=========================\nTesting MovieLens \'1 million\' dataset')
test('recsys-data/ratings-1m.dat', example_user=200, example_item=1000, k=10, minr=3, topn=10, cutoff=10, threshold=4, sep='::')
print('=========================\nDone.')

# Additional testing?
#student_test()

Testing toy 1 dataset
Reading the data at 11:06:36...
Ratings matrix takes 0.0 MB in RAM
--> elapsed time: 0:00:00 <--
-------------------------
Testing the ratings data structures
11 ratings by 4 users on 5 items
Ratings of user v: {'b': 4.0, 'c': 5.0, 'd': 3.0}
Ratings of item b: {'v': 4.0, 'x': 2.0, 'y': 4.0}
-------------------------
Testing RandomRecommender (top 4)
Four example recommendations:
    User v -> <a:0.9786 b:0.7992 d:0.7805 c:0.4615>
    User x -> <c:0.9447 a:0.6399 d:0.5218 e:0.4147>
    User y -> <b:0.7742 d:0.5684 c:0.4562 a:0.2646>
    User z -> <d:0.9437 e:0.6818 a:0.6176 c:0.6169>
Precision@4 = 0.0
Recall@4 = 0.0
--> elapsed time: 0:00:00 <--
-------------------------
Testing MajorityRecommender (top 4)
Four example recommendations:
    User v -> <b:2 c:2 d:1 e:1>
    User x -> <b:2 c:2 d:1 e:1>
    User y -> <b:2 c:2 d:1 e:1>
    User z -> <b:2 c:2 d:1 e:1>
Precision@4 = 0.0625
Recall@4 = 0.25
--> elapsed time: 0:00:00 <--
-------------------------
Testing Aver

/tmp/ipykernel_8400/2459295587.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.ratings_df=pd.read_csv(filepath_or_buffer=file, sep = sep)


Ratings matrix takes 170.8 MB in RAM
--> elapsed time: 0:00:09 <--
-------------------------
Testing the ratings data structures
1,000,209 ratings by 6,040 users on 3,706 items
Ratings of user 200: {8: 3.0, 170: 5.0, 940: 4.0, 1059: 5.0, 1127: 4.0, 1441: 4.0, 1605: 1.0, 1911: 2.0, 2041: 5.0, 2355: 3.0, 2555: 2.0, 2572: 5.0, 2599: 3.0, 2605: 2.0, 2683: 3.0, 2694: 5.0, 2699: 4.0, 2706: 2.0, 2759: 2.0, 2770: 2.0, 2779: 2.0, 2827: 3.0, 2997: 4.0, 3051: 5.0, 3408: 5.0, 3751: 4.0}
Ratings of item 1000: {474: 5.0, 1733: 2.0, 2820: 3.0, 3032: 4.0, 3193: 2.0, 3197: 3.0, 3224: 4.0, 3391: 3.0, 3507: 4.0, 3526: 3.0, 3644: 5.0, 3829: 4.0, 3868: 4.0, 3942: 1.0, 4088: 3.0, 4139: 4.0, 4732: 3.0, 5426: 2.0, 5682: 1.0, 5916: 1.0}
-------------------------
Testing RandomRecommender (top 10)
Four example recommendations:
    User 1 -> <3786:0.9996 3167:0.9991 136:0.999 1279:0.999 2558:0.9989 230:0.9988 1063:0.9987 2908:0.9987 313:0.9986 2643:0.9985>
    User 2 -> <2577:1 347:0.9995 3402:0.9994 2006:0.9986

### Salida obtenida por el estudiante

*(por hacer)*